In [1]:
from deeppavlov.core.models.component import Component
from deeppavlov.core.agent.agent import Agent
from deeppavlov.models.classifiers.tokens_matcher.tokens_matcher import TokensMatcher

/home/yoptar/reps/DeepPavlov/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to /home/yoptar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/yoptar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/yoptar/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/yoptar/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-06-18 16:03:17.920 DEBUG in 'gensim.models.doc2vec'['doc2vec'] at line 73: Fast versio

In [2]:
class SimpleSkill(Component):
    @staticmethod
    def _confidence_fn(utterances_batch):
        return [1 for _ in utterances_batch]  # by default always be confident
    
    def __init__(self, response, confidence_fn=None, *args, **kwargs):
        self.response = response
        if confidence_fn is not None:
            self._confidence_fn = confidence_fn
    
    def __call__(self, utterances_batch, history_batch, states_batch, *args, **kwargs):
        tokens = [u.lower().split() for u in utterances_batch]
        return [self.response for _ in utterances_batch], self._confidence_fn(tokens)

In [3]:
hello = SimpleSkill('Hello world!', confidence_fn=TokensMatcher(["hi", "hello", "good day"]))
bye = SimpleSkill('Good bye world!', confidence_fn=TokensMatcher(["bye", "chao", "see you"]))
other = SimpleSkill("I don't understand, sorry")

In [4]:
class CustomSelector(Component):
    def __init__(self, *args, **kwargs):
        pass
    
    def __call__(self, utterances, batch_history, *responses):
        responses, confidences = zip(*[zip(*r) for r in responses])
        indexes = [c.index(max(c)) for c in zip(*confidences)]
        return [responses[i] for i, *responses in zip(indexes, *responses)]

In [5]:
agent = Agent([hello, bye, other], skills_selector=CustomSelector())

In [6]:
agent(['Hello', 'Bye', 'Or not'])

['Hello world!', 'Good bye world!', "I don't understand, sorry"]